# コストの下限を求める


In [19]:
import os

import time
import datetime

import subprocess
import logging
from concurrent import futures

import numpy as np
import pandas as pd

In [20]:
TESTSET_DIR = os.path.join('/home', 'jupyter', 'work', '01_testset')
PRJ_DIR = os.path.join('/home', 'jupyter', 'work', '02_cost_lower_bound')

## 実行するロジックの指定

In [21]:
def solve(seed, N, M, D, K, problem, prog_path):
    start_time = time.perf_counter()
    
    command_str = 'echo {} | {}'.format(problem, prog_path)
    res = subprocess.run(command_str, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)

    # 経過時間(ミリ秒単位)
    e_time = time.perf_counter() - start_time
    e_time = int(1000 * e_time)    
    #print('{}'.format(prob_id))    
    return (seed, N, M, D, K, e_time, res)

In [22]:
def run_test(testset_name):
    result_df = pd.DataFrame()
    future_list = []

    logger.info('Start')
    
    testset_path = os.path.join(TESTSET_DIR, testset_name+'.csv')
    testset_df = pd.read_csv(testset_path)

    with futures.ThreadPoolExecutor(max_workers=12) as executor:
        for _, row in testset_df.iterrows():
            seed, N, M, D, K, problem = row

            # バッチ実行
            future = executor.submit(solve, seed=seed,  N=N, M=M, D=D, K=K, problem=problem, prog_path=prog_path)
            future_list.append(future)

        _ = futures.as_completed(fs=future_list)

    for future in future_list:
        seed, N, M, D, K, e_time, res = future.result()

        # 結果をまとめる
        solve_result = []
        
        solve_result.append(testset_name)

        # 問題パラメタ
        solve_result.append(seed)
        solve_result.append(N)
        solve_result.append(M)
        solve_result.append(K)
        solve_result.append(D)

        # 経過時間
        solve_result.append(e_time)

        try:
            # コストの下限
            cost_lb = int(res.stderr.decode('utf-8'))
                       
        except Exception as e:
            print('Error: seed={}'.format(seed))
            print(e)
            return

        solve_result.append(cost_lb)

        result_df = pd.concat([result_df, pd.DataFrame(solve_result).T], axis=0)

    logger.info('finish!')
    
    # 結果を整形
    result_df.index = range(result_df.shape[0])
    result_df.columns = ['testset', 'seed', 'N', 'M', 'D', 'K', 'time', 'cost_lb']

    return result_df

In [23]:
logger = logging.getLogger(__name__)

fmt = "%(asctime)s: %(message)s"
logging.basicConfig(level=logging.INFO, format=fmt)

In [24]:
PROG_NAME_LIST = ['main']
TEST_LIST = ['02_test_sys']
#TEST_LIST = ['01_test_pre']
#TEST_LIST = ['00_test_mini']

result_dict = {}
summary_all_dict = {}

for PROG_NAME in PROG_NAME_LIST:
    prog_path = os.path.join(PRJ_DIR, PROG_NAME)
    
    result_df = pd.DataFrame()
    
    for testset_name in TEST_LIST:
        testset_result_df = run_test(testset_name)
        result_df = pd.concat([result_df, testset_result_df], axis=0)

2023-01-28 17:31:43,816: Start
2023-01-28 19:28:04,353: finish!


In [25]:
result_df

,testset,seed,N,M,D,K,time,cost_lb
0,02_test_sys,2641,960,2496,156,28,104696,4140048
1,02_test_sys,5324,688,1610,345,7,30577,23015348
2,02_test_sys,972,532,1176,83,24,13188,7807268
3,02_test_sys,380,985,1770,91,28,79807,12382864
4,02_test_sys,5534,551,990,95,14,10644,25953257
...,...,...,...,...,...,...,...,...
1995,02_test_sys,548,647,1170,209,6,17864,56475388
1996,02_test_sys,3329,993,1886,167,19,58205,14002655
1997,02_test_sys,6329,608,1226,77,22,16288,10304208
1998,02_test_sys,1890,687,1222,97,21,16798,17641104


# 結果ログの保存

In [26]:
output_path = os.path.join(TESTSET_DIR, TEST_LIST[0] + '_cost_lb.csv')
result_df.to_csv(output_path, index=False)